In [1]:
import os 
os.chdir("/home/canyon/Bus-Weather-Impacts")
from src.utils import *
import pandas as pd
import os
import osmnx as ox
import numpy as np
import geopandas as gpd
import networkx as nx
from sklearn.neighbors import KDTree
import numpy as np
pd.options.mode.chained_assignment = None
pd.set_option('display.float_format', '{:.02f}'.format)
from geopy.distance import geodesic

In [4]:
GTFS_PATH = "https://urbantech-public.s3.amazonaws.com/DO-NOT-DELETE-BUSOBSERVATORY-PUBLIC-DATASET/one-system-day.tar.gz"
buses = read_parquet_from_tar_gz(GTFS_PATH)[["route", "trip_id", "timestamp", "lat", "lon"]]

In [5]:
buses = gpd.GeoDataFrame(
        buses,
        geometry=gpd.points_from_xy(buses["lon"], buses["lat"]),
        crs=4326
    )

buses = buses.to_crs(2263)
buses["planar_x"] = buses.geometry.x
buses["planar_y"] = buses.geometry.y
buses = buses.sort_values(["trip_id", "timestamp"])

In [6]:
buses["planar_x_prev"] = buses["planar_x"].shift(1)
buses["planar_y_prev"] = buses["planar_y"].shift(1)
buses["time_diff_seconds"] = buses.groupby("trip_id")["timestamp"].diff().dt.total_seconds()
buses = buses.query("time_diff_seconds < 70 and time_diff_seconds > 50")

In [7]:
buses["distance"] = np.sqrt((buses["planar_x_prev"] - buses["planar_x"])**2 +  (buses["planar_y_prev"] - buses["planar_y"])**2) 
buses["speed_mph"] = (buses["distance"] / 5280) / (buses["time_diff_seconds"] / 3600)
buses["speed_mph"].describe()
buses_trim = buses[["route", "trip_id", "timestamp", "speed_mph", "geometry"]]